# 姿勢推定の実装

- 本ファイルでは、学習させたOpenPoseで姿勢推定を行います。


検出部位（関節点）
0:鼻
1:首
2:右肩
3:右肘
4:右手首
5:左肩
6:左肘
7:左手首
8:右尻
9:右膝
10:右足首
11:左尻
12:左膝
13:左足首
14:右目
15:左目
16:右耳
17:左耳

In [ ]:
#ライブラリをインポート
from PIL import Image
import cv2
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
%matplotlib inline

import torch

### モデルの定義

In [ ]:
from utils.openpose_net import OpenPoseNet

# 学習済みモデルと本章のモデルでネットワークの層の名前が違うので、対応させてロードする
# モデルの定義
net = OpenPoseNet()

# 学習済みパラメータをロードする
net_weights = torch.load(
    './weights/pose_model.pth', map_location={'cuda:0': 'cpu'})
keys = list(net_weights.keys())

weights_load = {}

# ロードした内容を、構築したモデルのパラメータ名net.state_dict().keys()にコピーする
for i in range(len(keys)):
    weights_load[list(net.state_dict().keys())[i]
                 ] = net_weights[list(keys)[i]]

# コピーした内容をモデルに与える
state = net.state_dict()
state.update(weights_load)
net.load_state_dict(state)

print('ネットワーク設定完了：学習済みの重みをロードしました')


### データの準備

In [ ]:
# 画像を読み込み、前処理します

test_image = './data/image.jpg'  #宿題ではここに集合写真のパスを指定してください
oriImg = cv2.imread(test_image)  # B,G,Rの順番

# BGRをRGBにして表示
oriImg = cv2.cvtColor(oriImg, cv2.COLOR_BGR2RGB)
plt.imshow(oriImg)
plt.show()

# 画像のリサイズ
size = (368, 368)
img = cv2.resize(oriImg, size, interpolation=cv2.INTER_CUBIC)

# 画像の前処理
img = img.astype(np.float32) / 255.

# 色情報の標準化
color_mean = [0.485, 0.456, 0.406]
color_std = [0.229, 0.224, 0.225]

preprocessed_img = img.copy()  # RGB

for i in range(3):
    preprocessed_img[:, :, i] = preprocessed_img[:, :, i] - color_mean[i]
    preprocessed_img[:, :, i] = preprocessed_img[:, :, i] / color_std[i]

# （高さ、幅、色）→（色、高さ、幅）
img = preprocessed_img.transpose((2, 0, 1)).astype(np.float32)

# 画像をTensorに
img = torch.from_numpy(img)

# ミニバッチ化：torch.Size([1, 3, 368, 368])
x = img.unsqueeze(0)


### Confidence MapとPAFsを求める

In [ ]:

net.eval()
predicted_outputs, _ = net(x)

# 画像をテンソルからNumPyに変化し、サイズを戻します
pafs = predicted_outputs[0][0].detach().numpy().transpose(1, 2, 0)
heatmaps = predicted_outputs[1][0].detach().numpy().transpose(1, 2, 0)

pafs = cv2.resize(pafs, size, interpolation=cv2.INTER_CUBIC)
heatmaps = cv2.resize(heatmaps, size, interpolation=cv2.INTER_CUBIC)

pafs = cv2.resize(
    pafs, (oriImg.shape[1], oriImg.shape[0]), interpolation=cv2.INTER_CUBIC)
heatmaps = cv2.resize(
    heatmaps, (oriImg.shape[1], oriImg.shape[0]), interpolation=cv2.INTER_CUBIC)

print('shape : (高さ, 幅, クラス数)')
print('PAFsのshape : ' + str(pafs.shape))
print('Confidence Mapのshape : ' + str(heatmaps.shape))

# PAFsのクラス数はリンク19本のx,y方向のベクトル座標
# Confidence Mapのクラス数は関節数（18個）といずれでもないクラスの合計
# 出力配列の値は，各ピクセルが各クラスである確信度（≒確率）に対応

### confidence mapとPAFsを可視化

In [ ]:
# 左膝と左足首のconfidence map(heatmap)、そして左膝と左足首をつなぐPAFのxベクトルを可視化する
# 左肘
heat_map = heatmaps[:, :, 12]  # 6は左膝
heat_map = Image.fromarray(np.uint8(cm.jet(heat_map)*255))
heat_map = np.asarray(heat_map.convert('RGB'))

# 合成して表示
print('左膝のConfidence Map')
blend_img = cv2.addWeighted(oriImg, 0.5, heat_map, 0.5, 0)
plt.imshow(blend_img)
plt.show()


# 左足首
heat_map = heatmaps[:, :, 13]  # 7は左足首
heat_map = Image.fromarray(np.uint8(cm.jet(heat_map)*255))
heat_map = np.asarray(heat_map.convert('RGB'))

# 合成して表示
print('左足首のConfidence Map')
blend_img = cv2.addWeighted(oriImg, 0.5, heat_map, 0.5, 0)
plt.imshow(blend_img)
plt.show()


# 左膝と左足首をつなぐPAFのxベクトル
paf = pafs[:, :, 10]
paf = Image.fromarray(np.uint8(cm.jet(paf)*255))
paf = np.asarray(paf.convert('RGB'))

# 合成して表示
print('左膝と左足首をつなぐPAFのxベクトル')
blend_img = cv2.addWeighted(oriImg, 0.5, paf, 0.5, 0)
plt.imshow(blend_img)
plt.show()

# 左膝と左足首をつなぐPAFのyベクトル
paf = pafs[:, :, 11]
paf = Image.fromarray(np.uint8(cm.jet(paf)*255))
paf = np.asarray(paf.convert('RGB'))

# 合成して表示
print('左膝と左足首をつなぐPAFのyベクトル')
blend_img = cv2.addWeighted(oriImg, 0.5, paf, 0.5, 0)
plt.imshow(blend_img)
plt.show()


### 姿勢の構成

In [ ]:
from utils.decode_pose import decode_pose
_, result_img, _, _ = decode_pose(oriImg, heatmaps, pafs)
#decode_pose : 各部位をつなぐリンクを求める関数


In [ ]:
# 結果を描画
plt.imshow(oriImg)
plt.show()

plt.imshow(result_img)
plt.savefig("HPE_result.png")  #姿勢推定結果を保存
plt.show()


以上